<a href="https://colab.research.google.com/github/3000minsu/2023-LG-Aimers/blob/main/1_imputation_by_simple_imputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i]) 
print('Done.')
train
test

Done.


,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,5,2,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,5,2,2.0,91.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,4,2,2.0,96.0,0.0,45.0,11.0,0.0,45.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,5,2,2.0,91.0,0.0,45.0,10.0,0.0,50.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,5,2,2.0,95.0,0.0,45.0,10.0,0.0,51.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#그래서 하나의 세트에서 모두 결측치인 열은 두 세트에서 모두 삭제 
#data leakage 인가?

invalid=[]
for col in train.columns:
  if(train[col].isnull().all()):
    invalid+=[col]

train = pd.concat([train,test], ignore_index=True) #인덱스 초기화
train = train.drop(columns=invalid)
tmp=train
train = tmp.iloc[:598,:]
test = tmp.iloc[598:,:]

#진또배기 열들 리스트 저장장
column_set=[]
for col in train.columns:
  column_set+=[col]

print(test.shape)
print(train.shape)


(310, 2795)
(598, 2795)


In [ ]:
from sklearn.impute import SimpleImputer
# strategy 바꾸면서 평균갑, 중앙값, 최빈값, 정해진 값 설정 가능능
imputer = SimpleImputer(strategy= 'median')
train=imputer.fit_transform(train)

k = 0
test = pd.DataFrame(test)
train = pd.DataFrame(train)
medians = train.median()
for i in column_set:
  test[[i]] = test[[i]].fillna(medians[k]) 
  k= k+1

print(test.iloc[:, 350])
medians[350]

598    0.00010
599    0.00010
600    0.00010
601    0.00011
602    0.00011
        ...   
903    0.00010
904    0.00010
905    0.00010
906    0.00010
907    0.00010
Name: X_349, Length: 310, dtype: float64


0.0001

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ss.fit(train)
train_scaled=ss.transform(train)
test_scaled=ss.transform(test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
RF = RandomForestClassifier(random_state=37).fit(train_scaled, train_y)
print('Done.')

Done.


In [ ]:
preds = RF.predict(test_scaled)
print('Done.')

Done.


In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
submit['Y_Class'] = preds

In [ ]:
submit.to_csv('./baseline_submission.csv', index=False)